In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
label = 'rate'
mypath = 'data/vision/train/'

In [3]:
colors = {
"pink" : 1,
"purple":2,
"yellow" :3,
"orange":4,
"white":5,
"silver":6,
"grey":7,
"black":8,
"red":9,
"brown":10,
"green":11,
"blue":12
}
input_dim = 224
n_channel = 3

In [4]:
len(colors)

12

In [5]:
from os import listdir
from os.path import isfile, join , isdir
onlyfolders = [f for f in listdir(mypath) if isdir(join(mypath, f))]

In [6]:
classes = onlyfolders

In [7]:
from PIL import Image
image = Image.open('data/vision/train/black/105184860.jpg')
image.size

(224, 224)

In [ ]:
np.array(image).shape

In [84]:
images = []
labels = []

In [85]:
for d in onlyfolders:
    path = "{}/{}/".format(mypath , d)
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    for f in onlyfiles:
        images.append(np.array(Image.open(f)))
        labels.append(colors[d])

In [10]:
X = np.array(images)

In [11]:
y = np.array(labels)

In [12]:
# X=np.rollaxis(X, 3, 1)

In [13]:
X.shape , y.shape

((6239, 224, 224, 3), (6239,))

In [86]:
for i , ix in enumerate(images):
    images[i] = ix.flatten()
Xf = np.array(images)
Xf.shape

(6239, 150528)

In [83]:
images[0]

<function ndarray.flatten>

In [88]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(Xf, np.array(labels))


c:\users\lion\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(8,), random_state=1 , verbose=True)

In [ ]:
clf.fit(X,y)

In [90]:
clf.predict_proba(Xf).shape

(6239, 12)

In [89]:
clf.score(Xf , y)

0.7300849495111396

## neural net

In [ ]:
X=X.astype('float32')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33, random_state=42)

In [ ]:
batch_size=64

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:


class LeNET(nn.Module):
    def __init__(self , input_dim):
        super(LeNET,self).__init__()
        self.conv_model = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5),   # (N,3,224,224) --> (N,6,220,220) 
                                        nn.Tanh(),                                               # TanH --> Activation Function
                                        nn.AvgPool2d(kernel_size=2,stride=2),                    # (N,6,220,220) --> (N,6,110,110)
                                        nn.Conv2d(6,16,5),nn.Tanh(),                             # (N,6,110,110) --> (N,16,106,106)
                                        nn.AvgPool2d(2,stride=2),                                # (N,16,106,106) --> (N,16,52,52)
                                        nn.Conv2d(16,32,5),nn.Tanh(), 
                                        nn.AvgPool2d(2,stride=2))

        #Dense Layer
        self.dense_layer = nn.Sequential(nn.Linear(in_features=18432,out_features=120),           #16*5*5 = 400 as input
                                         nn.Tanh(),
                                         nn.Linear(120,84),
                                         nn.Tanh(),
                                     nn.Linear(84,13))
    def forward(self,x):
        y = self.conv_model(x)
        #flatten the result from Conv model
        y = torch.flatten(y,1) # 1 --> dimension (N,16,5,5)
        y = self.dense_layer(y)
        return y

In [ ]:
net = LeNET(input_dim).cuda()

In [ ]:
def model_eval(X ,y ,net , batch_size):
    total = 0
    correct = 0
    for i in range(0,len(X)//batch_size):
        images = torch.from_numpy(X[i*batch_size:(i+1)*batch_size]).cuda()
        lables = torch.from_numpy(y[i*batch_size:(i+1)*batch_size]).cuda()
        out = net(images)
        max_val, preds = torch.max(out,dim=1)
        #accuracy = (correctly predicted class / total testing class) × 100%
        total += lables.shape[0]                   # class value 
        correct += (preds == lables).sum().item()  # to summ correct in the batch then sum all of it
    accuracy = (100 * correct)/total

    return accuracy

In [ ]:

out = net(torch.from_numpy(X[:10]))
print(out.shape)

In [ ]:
loss_func(out , torch.tensor([1 for i in range(10)]))

In [ ]:
loss_func(out , torch.tensor(torch.from_numpy(y_train[:10]),dtype=torch.long))

In [ ]:
labels
out

In [ ]:
# defining Loss function
loss_func = nn.CrossEntropyLoss()
# defining optimizer
optimizer = torch.optim.Adam(net.parameters()) 

In [ ]:
y_train = y_train.astype('int')

In [ ]:
image_data.shape

In [ ]:
net(torch.rand(image_data.shape).cuda())

In [ ]:
epoch = 100

for j in range(epoch):

    for i in range(0,len(X_train)//batch_size):
        image_data = torch.from_numpy(X_train[i*batch_size:(i+1)*batch_size]).cuda()
        
        labels = torch.tensor(torch.from_numpy(y_train[i*batch_size:(i+1)*batch_size]) ,dtype=torch.long).cuda()
        #print(i,image_data.shape)
        
#         print(image_data)
        out = net(image_data) #image_data --> input data
        loss = loss_func(out , labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_acc = model_eval(X_train,y_train , net , batch_size) 
    test_acc  = model_eval(X_test,y_test , net , batch_size)

    print("Epoch :",j," train acc : "  ,train_acc , "test acc : " , test_acc)

In [ ]:
train_acc = model_eval(X_train,y_train , net , 32) 

In [ ]:
train_acc

## test

In [14]:
mypathtest = 'data/vision/test/'
imagestest =[]
ids = []

In [15]:

onlyfiles = [f for f in listdir(mypathtest) if isfile(join(mypathtest, f))]
for f in onlyfiles:
    imagestest.append(np.array(Image.open(join(mypathtest, f))))
    ids.append(f)

In [96]:
for i in range(len(imagestest)):
    imagestest[i] = imagestest[i].flatten()

In [97]:
Xtf = np.array(imagestest)
Xtf.shape

(2673, 150528)

In [17]:
Xt = np.array(imagestest)

In [18]:
Xt.shape

(2673, 224, 224, 3)

In [19]:
pred = clf.predict(Xt)

NameError: name 'clf' is not defined

In [ ]:
dfs = pd.DataFrame.from_dict({'file_name' : ids , 'color_id' : pred})
dfs.head()

In [ ]:
dfs.sort_values('file_name')

In [ ]:
dfs.to_csv('data/vision/ans1.csv' , index=False)

## keras

In [20]:
from keras.models import Sequential,Model,load_model
from keras.optimizers import SGD
from keras.layers import BatchNormalization, Lambda, Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.merge import Concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import numpy as np
import keras.backend as K

In [21]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17094746867180398020
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4510031374925429415
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4305363208865209104
physical_device_desc: "device: XLA_GPU device"
]


In [ ]:
with tf.device('/device:GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

In [22]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
import keras
import tensorflow as tf


tf.config.experimental.list_physical_devices('GPU')

In [23]:
def color_net(num_classes):
    # placeholder for input image
    input_image = Input(shape=(input_dim,input_dim,3))
    # ============================================= TOP BRANCH ===================================================
    # first top convolution layer
    top_conv1 = Convolution2D(filters=48,kernel_size=(11,11),strides=(4,4),
                              input_shape=(input_dim,input_dim,3),activation='relu')(input_image)
    top_conv1 = BatchNormalization()(top_conv1)
    top_conv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_conv1)

    # second top convolution layer
    # split feature map by half
    top_top_conv2 = Lambda(lambda x : x[:,:,:,:24])(top_conv1)
    top_bot_conv2 = Lambda(lambda x : x[:,:,:,24:])(top_conv1)

    top_top_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv2)
    top_top_conv2 = BatchNormalization()(top_top_conv2)
    top_top_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_top_conv2)

    top_bot_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv2)
    top_bot_conv2 = BatchNormalization()(top_bot_conv2)
    top_bot_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_bot_conv2)

    # third top convolution layer
    # concat 2 feature map
    top_conv3 = Concatenate()([top_top_conv2,top_bot_conv2])
    top_conv3 = Convolution2D(filters=192,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_conv3)

    # fourth top convolution layer
    # split feature map by half
    top_top_conv4 = Lambda(lambda x : x[:,:,:,:96])(top_conv3)
    top_bot_conv4 = Lambda(lambda x : x[:,:,:,96:])(top_conv3)

    top_top_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv4)
    top_bot_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv4)

    # fifth top convolution layer
    top_top_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv4)
    top_top_conv5 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_top_conv5) 

    top_bot_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv4)
    top_bot_conv5 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_bot_conv5)

    # ============================================= TOP BOTTOM ===================================================
    # first bottom convolution layer
    bottom_conv1 = Convolution2D(filters=48,kernel_size=(11,11),strides=(4,4),
                              input_shape=(227,227,3),activation='relu')(input_image)
    bottom_conv1 = BatchNormalization()(bottom_conv1)
    bottom_conv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_conv1)

    # second bottom convolution layer
    # split feature map by half
    bottom_top_conv2 = Lambda(lambda x : x[:,:,:,:24])(bottom_conv1)
    bottom_bot_conv2 = Lambda(lambda x : x[:,:,:,24:])(bottom_conv1)

    bottom_top_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv2)
    bottom_top_conv2 = BatchNormalization()(bottom_top_conv2)
    bottom_top_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_top_conv2)

    bottom_bot_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv2)
    bottom_bot_conv2 = BatchNormalization()(bottom_bot_conv2)
    bottom_bot_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_bot_conv2)

    # third bottom convolution layer
    # concat 2 feature map
    bottom_conv3 = Concatenate()([bottom_top_conv2,bottom_bot_conv2])
    bottom_conv3 = Convolution2D(filters=192,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_conv3)

    # fourth bottom convolution layer
    # split feature map by half
    bottom_top_conv4 = Lambda(lambda x : x[:,:,:,:96])(bottom_conv3)
    bottom_bot_conv4 = Lambda(lambda x : x[:,:,:,96:])(bottom_conv3)

    bottom_top_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv4)
    bottom_bot_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv4)

    # fifth bottom convolution layer
    bottom_top_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv4)
    bottom_top_conv5 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_top_conv5) 

    bottom_bot_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv4)
    bottom_bot_conv5 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_bot_conv5)

    # ======================================== CONCATENATE TOP AND BOTTOM BRANCH =================================
    conv_output = Concatenate()([top_top_conv5,top_bot_conv5,bottom_top_conv5,bottom_bot_conv5])

    # Flatten
    flatten = Flatten()(conv_output)

    # Fully-connected layer
    FC_1 = Dense(units=4096, activation='relu')(flatten)
    FC_1 = Dropout(0.6)(FC_1)
    FC_2 = Dense(units=4096, activation='relu')(FC_1)
    FC_2 = Dropout(0.6)(FC_2)
    output = Dense(units=num_classes, activation='softmax')(FC_2)
    
    model = Model(inputs=input_image,outputs=output)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    # sgd = SGD(lr=0.01, momentum=0.9, decay=0.0005, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [24]:
from keras.utils.np_utils import to_categorical
categorical_labels = to_categorical(labels, num_classes=13)
categorical_labels.shape

(6239, 13)

In [25]:
# initialise model
num_classes = len(colors)+1
model = color_net(num_classes)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 54, 54, 48)   17472       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 54, 54, 48)   17472       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 54, 54, 48)   192         conv2d[0][0]                     
______________________________________________________________________________________________

In [27]:
model.load_weights('color_model.h5')

In [ ]:
test_datagen = ImageDataGenerator(      
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.3,)

In [54]:
advtx , advty = [] , []
hard_to_guess = [1,5,6,7,10]
n=500
for ha in hard_to_guess:
    jc=0
    for i , la in enumerate(labels):
        if(la ==ha):
            advtx.append(images[i])
            advty.append(la)
            jc+=1
        if(jc>=n):
            break

In [ ]:
plt.hist(advty)

In [55]:
advX = np.array(advtx)
advY = np.array(advty)

In [ ]:
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

In [118]:
sgd = SGD(lr=1e-8, decay=1e-6, momentum=0.9, nesterov=True)

In [119]:
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [120]:
img_rows , img_cols = input_dim,input_dim
num_classes = len(colors)+1
batch_size = 256
nb_epoch = 10

# # initialise model
# model = color_net(num_classes)

filepath = 'color_weights.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

train_datagen = ImageDataGenerator( 
    zca_whitening=True,
#         shear_range=0.2,
#         zoom_range=0.3,
        horizontal_flip=True,
#         rotation_range=45,
        vertical_flip=True,
        validation_split=0.2)
test_datagen = ImageDataGenerator(        
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.3)

# training_set = train_datagen.flow_from_directory(
#             'train/',
#             target_size=(img_rows, img_cols),
#             batch_size=batch_size,
#             class_mode='categorical')
# test_set = test_datagen.flow_from_directory(
#             'test/',
#             target_size=(img_rows, img_cols),
#             batch_size=batch_size,
#             class_mode='categorical')
# training_set = train_datagen.flow_from_dataframe(X_train,
#             target_size=(img_rows, img_cols),
#             batch_size=batch_size,
#             class_mode='categorical')
# training_set = train_datagen.(X_test,
#             target_size=(img_rows, img_cols),
#             batch_size=batch_size,
#             class_mode='categorical')


# model.fit_generator(
#         training_set,
#         steps_per_epoch=12000,
#         epochs=nb_epoch,
#         validation_data=test_set,
#         validation_steps=3000,
#         callbacks=callbacks_list)
with tf.device('/device:GPU:0'):
#     for i in range(10):
#     model.fit(X , categorical_labels , batch_size=batch_size,epochs = nb_epoch , validation_split=0.1)
    model.fit(train_datagen.flow(X , categorical_labels , batch_size), steps_per_epoch=len(X) / batch_size, epochs=nb_epoch)
#     model.fit(advX , to_categorical(advY, num_classes=13),batch_size=batch_size,epochs = 1)
# model.save('color_model.h5')

C:\Users\Lion\AppData\Roaming\Python\Python37\site-packages\keras_preprocessing\image\image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Lion\AppData\Roaming\Python\Python37\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Lion\AppData\Roaming\Python\Python37\site-packages\keras_preprocessing\image\image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/10
25/24 [==============================] - 62s 2s/step - loss: 0.1776 - accuracy: 0.9377
Epoch 2/10
25/24 [==============================] - 62s 2s/step - loss: 0.1775 - accuracy: 0.9368
Epoch 3/10
25/24 [==============================] - 62s 2s/step - loss: 0.1732 - accuracy: 0.9388
Epoch 4/10
25/24 [==============================] - 62s 2s/step - loss: 0.1787 - accuracy: 0.9397
Epoch 5/10
25/24 [==============================] - 64s 3s/step - loss: 0.1754 - accuracy: 0.9399
Epoch 6/10
25/24 [==============================] - 65s 3s/step - loss: 0.1671 - accuracy: 0.9461
Epoch 7/10
25/24 [==============================] - 65s 3s/step - loss: 0.1775 - accuracy: 0.9396
Epoch 8/10
25/24 [==============================] - 65s 3s/step - loss: 0.1838 - accuracy: 0.9375
Epoch 9/10
25/24 [==============================] - 63s 3s/step - loss: 0.1820 - accuracy: 0.9389
Epoch 10/10
25/24 [==============================] - 62s 2s/step - loss: 0.1737 - accuracy: 0.9429


In [121]:
score, acc = model.evaluate(X , categorical_labels ,
                            batch_size=batch_size)
model.save('color_model.h5')
print('Test score:', score)
print('Test accuracy:', acc)

25/25 [==============================] - 10s 407ms/step - loss: 0.0977 - accuracy: 0.9686
Test score: 0.09769605845212936
Test accuracy: 0.9685847163200378


In [ ]:
labels[0]

In [122]:
pred = model.predict(Xt)

In [98]:
pred2 = clf.predict_proba(Xtf)
pred2.shape

(2673, 12)

In [104]:

pred3=np.concatenate((np.zeros((2673,1)) , pred2) , axis=1)
pred3[0]

array([0.00000000e+00, 6.63596730e-04, 1.17566198e-02, 5.74800747e-04,
       2.37803003e-04, 1.20431150e-04, 3.80468230e-02, 2.22868838e-02,
       2.07084426e-01, 4.34944910e-03, 6.84231341e-01, 1.24883155e-02,
       1.81595106e-02])

In [105]:
pred3

array([[0.00000000e+00, 6.63596730e-04, 1.17566198e-02, ...,
        6.84231341e-01, 1.24883155e-02, 1.81595106e-02],
       [0.00000000e+00, 9.74664763e-02, 3.61413584e-01, ...,
        1.79332862e-04, 1.27340282e-09, 2.57023526e-05],
       [0.00000000e+00, 1.24218124e-06, 2.62649591e-14, ...,
        3.70154552e-04, 1.12010824e-02, 1.71457825e-16],
       ...,
       [0.00000000e+00, 5.96794073e-03, 4.57123850e-02, ...,
        7.01172528e-02, 1.26265795e-01, 1.05115168e-01],
       [0.00000000e+00, 2.47453554e-03, 1.61118447e-02, ...,
        7.62951164e-02, 5.84689033e-01, 1.51730610e-02],
       [0.00000000e+00, 1.09937860e-03, 6.76537304e-03, ...,
        6.66520706e-03, 5.28005955e-01, 9.98602088e-02]])

In [113]:
pred = 0.88*pred + 0.12*pred3

In [123]:
pred = np.apply_along_axis(np.argmax ,1, pred)

In [75]:
pred

array([10,  2, 11, ...,  7,  7, 11], dtype=int64)

In [124]:
dfs = pd.DataFrame.from_dict({'file_name' : ids , 'color_id' : pred})
dfs.head()

,file_name,color_id
0,1000000004.jpg,10
1,1000000033.jpg,2
2,1000000678.jpg,11
3,1000001233.jpg,2
4,1000001547.jpg,11


In [125]:
dfs.sample(10)

,file_name,color_id
2306,117159660.jpg,3
1852,117075364.jpg,5
926,115254638.jpg,9
2504,117216927.jpg,11
2547,117224412.jpg,2
211,105344084.jpg,8
954,115260418.jpg,5
2290,117152272.jpg,12
1951,117093136.jpg,1
2140,117117574.jpg,5


In [ ]:
dfs.sort_values('file_name')

In [35]:
colors

{'pink': 1,
 'purple': 2,
 'yellow': 3,
 'orange': 4,
 'white': 5,
 'silver': 6,
 'grey': 7,
 'black': 8,
 'red': 9,
 'brown': 10,
 'green': 11,
 'blue': 12}

In [126]:
dfs.to_csv('data/vision/ans2.csv' , index=False)